In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']

In [3]:
class HeavierSimpleCNN(nn.Module):
    def __init__(self):
        super(HeavierSimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 2 * 2, 1024)  # Adjust the in_features based on the last layer output
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(x.size(0), -1)  # Ensure that the batch size is maintained
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [4]:
# Transforms and DataLoader setup
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [5]:
# Dataset and DataLoader
train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

In [6]:
train_loader = DataLoader(train_data, batch_size=20, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=20, shuffle=False, drop_last=True)

In [7]:
# Initialize the model, loss criterion and optimizer
model = HeavierSimpleCNN()
if torch.cuda.is_available():
    model.cuda()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {running_loss / len(train_loader.dataset):.4f}')


In [10]:
# Function to evaluate the model
def evaluate_model(model, val_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

In [11]:
# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, val_loader)

Epoch 1/50 - Loss: 1.1946
Epoch 2/50 - Loss: 0.7036
Epoch 3/50 - Loss: 0.6930
Epoch 4/50 - Loss: 0.7129
Epoch 5/50 - Loss: 0.6410
Epoch 6/50 - Loss: 0.6612
Epoch 7/50 - Loss: 0.6484
Epoch 8/50 - Loss: 0.6598
Epoch 9/50 - Loss: 0.6531
Epoch 10/50 - Loss: 0.6331
Epoch 11/50 - Loss: 0.6399
Epoch 12/50 - Loss: 0.6362
Epoch 13/50 - Loss: 0.6443
Epoch 14/50 - Loss: 0.6517
Epoch 15/50 - Loss: 0.6481
Epoch 16/50 - Loss: 0.6396
Epoch 17/50 - Loss: 0.6306
Epoch 18/50 - Loss: 0.6323
Epoch 19/50 - Loss: 0.6274
Epoch 20/50 - Loss: 0.6311
Epoch 21/50 - Loss: 0.6373
Epoch 22/50 - Loss: 0.6285
Epoch 23/50 - Loss: 0.6259
Epoch 24/50 - Loss: 0.6445
Epoch 25/50 - Loss: 0.6278
Epoch 26/50 - Loss: 0.6032
Epoch 27/50 - Loss: 0.6070
Epoch 28/50 - Loss: 0.6382
Epoch 29/50 - Loss: 0.6126
Epoch 30/50 - Loss: 0.6270
Epoch 31/50 - Loss: 0.6154
Epoch 32/50 - Loss: 0.6120
Epoch 33/50 - Loss: 0.6305
Epoch 34/50 - Loss: 0.6062
Epoch 35/50 - Loss: 0.6081
Epoch 36/50 - Loss: 0.5741
Epoch 37/50 - Loss: 0.5800
Epoch 38/5